In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
np.set_printoptions(suppress=True)

In [3]:
#To indicate how to use these libraries, we first create a linear model from random data.
rng = np.random.default_rng(seed=12345)

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size= size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
         dnorm(0, 0.6, size=N),
         dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta=[0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [4]:
#we wrote down the "true" model with known parameters beta. Here, we created a custom function dnorm
#dnorm creates a helper function to generate normally distributed data with a particular mean and variance.
X[:5]

array([[-0.90050602, -0.18942958, -1.0278702 ],
       [ 0.79925205, -1.54598388, -0.32739708],
       [-0.55065483, -0.12025429,  0.32935899],
       [-0.16391555,  0.82403985,  0.20827485],
       [-0.04765129, -0.21314698, -0.04824364]])

In [5]:
y[:5]

array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445])

In [6]:
#a linear model is generally fitted with an intercept term like in patsy with 1.
#We will use sm.add_constant to add an intercept term here.
X_model = sm.add_constant(X)

In [7]:
X_model[:5]

array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899],
       [ 1.        , -0.16391555,  0.82403985,  0.20827485],
       [ 1.        , -0.04765129, -0.21314698, -0.04824364]])

In [8]:
y[:5]

array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445])

In [9]:
#sm.OLS class can fit an ordinary least squares linear regression
model = sm.OLS(y, X)

In [10]:
results = model.fit()

In [11]:
results.params

array([0.06681503, 0.26803235, 0.45052319])

In [12]:
#summary method on results can print a model with diagnostic output of the model
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Mon, 06 Feb 2023   Prob (F-statistic):                    2.66e-13
Time:                        08:38:38   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [13]:
#In our previous example, we had generic names x1, x2, etc
#What if we had model parameters in a DataFrame
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

In [14]:
data[:5]

,col0,col1,col2
0,-0.900506,-0.189430,-1.027870
1,0.799252,-1.545984,-0.327397
2,-0.550655,-0.120254,0.329359
3,-0.163916,0.824040,0.208275
4,-0.047651,-0.213147,-0.048244


In [15]:
data['y'] = y

In [16]:
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [17]:
#Now, we can use the statsmodels formula.
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

In [18]:
results

In [19]:
results.params

Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [20]:
results.tvalues
#statsmodels returns results as Series with the DataFrame names attached.

Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64